In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path


from selenium import webdriver #selenium 4.20.0
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.common.exceptions import WebDriverException

from webdriver_manager.chrome import ChromeDriverManager
 # version 4.0.1

In [3]:
options = webdriver.ChromeOptions()
options.set_capability(
    "goog:loggingPrefs", {"performance": "ALL", "browser": "ALL"}
)

# Make sure you already have Chrome installed
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

In [4]:
driver.set_page_load_timeout(30)

try:
    driver.get("https://www.sofascore.com/spain-france/GObsYTb#id:11874020")
except:
    pass


driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [9]:
# extract requests from logs
logs_raw = driver.get_log("performance")
logs = [json.loads(lr["message"])["message"] for lr in logs_raw]

In [11]:
desired_keywords = ['shotmap']

for log in logs:
    # Check if the log contains the 'method' key
    if 'method' in log:
        # Check if the log method is 'Network.responseReceived'
        if log["method"] == "Network.responseReceived":
            # Extract request ID and URL
            request_id = log["params"]["requestId"]
            url = log["params"]["response"]["url"]
            # Check if the URL contains any of the desired keywords
            if any(keyword in url for keyword in desired_keywords):
                # Print the request ID and URL
                print(f"Request ID: {request_id}, URL: {url}")
                # # Try to retrieve the response body using the request ID
                try:
                    response = driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': request_id})
                    stats = response['body']
                    if stats:
                        print("Response body successfully retrieved:")
                        print(stats)
                        break
                    else:
                        print("Response body is empty. Trying next request ID...")
                except Exception as e:
                    print(f"Error occurred while retrieving response for request ID {request_id}: {e}")
                    continue

# stats

Request ID: 9832.3283, URL: https://www.sofascore.com/api/v1/event/11874020/shotmap
Response body successfully retrieved:
{"shotmap":[{"player":{"name":"Antoine Griezmann","slug":"antoine-griezmann","shortName":"A. Griezmann","position":"F","jerseyNumber":"7","userCount":110394,"id":85859,"fieldTranslations":{"nameTranslation":{"ar":"\u063a\u0631\u064a\u0632\u0645\u0627\u0646, \u0627\u0646\u0637\u0648\u0627\u0646"},"shortNameTranslation":{"ar":"\u0627. \u063a\u0631\u064a\u0632\u0645\u0627\u0646"}}},"isHome":false,"shotType":"block","situation":"assisted","playerCoordinates":{"x":10.5,"y":58.3,"z":0},"bodyPart":"head","goalMouthLocation":"low-centre","goalMouthCoordinates":{"x":0,"y":49.4,"z":19},"blockCoordinates":{"x":9,"y":57.2,"z":0},"xg":0.02813040651381,"id":3433360,"time":90,"addedTime":5,"timeSeconds":5679,"draw":{"start":{"x":58.3,"y":10.5},"block":{"x":57.2,"y":9},"end":{"x":50.6,"y":0},"goal":{"x":50.6,"y":81}},"reversedPeriodTime":1,"reversedPeriodTimeSeconds":621,"incidentT

In [12]:
data = json.loads(stats)
data

{'shotmap': [{'player': {'name': 'Antoine Griezmann',
    'slug': 'antoine-griezmann',
    'shortName': 'A. Griezmann',
    'position': 'F',
    'jerseyNumber': '7',
    'userCount': 110394,
    'id': 85859,
    'fieldTranslations': {'nameTranslation': {'ar': 'غريزمان, انطوان'},
     'shortNameTranslation': {'ar': 'ا. غريزمان'}}},
   'isHome': False,
   'shotType': 'block',
   'situation': 'assisted',
   'playerCoordinates': {'x': 10.5, 'y': 58.3, 'z': 0},
   'bodyPart': 'head',
   'goalMouthLocation': 'low-centre',
   'goalMouthCoordinates': {'x': 0, 'y': 49.4, 'z': 19},
   'blockCoordinates': {'x': 9, 'y': 57.2, 'z': 0},
   'xg': 0.02813040651381,
   'id': 3433360,
   'time': 90,
   'addedTime': 5,
   'timeSeconds': 5679,
   'draw': {'start': {'x': 58.3, 'y': 10.5},
    'block': {'x': 57.2, 'y': 9},
    'end': {'x': 50.6, 'y': 0},
    'goal': {'x': 50.6, 'y': 81}},
   'reversedPeriodTime': 1,
   'reversedPeriodTimeSeconds': 621,
   'incidentType': 'shot'},
  {'player': {'name': 'Kyli

In [13]:
# Parse the JSON response
data = json.loads(stats)
data

df_players = pd.json_normalize(data['shotmap'])
df_players['Team'] = np.where(df_players['isHome'] == True, 'Home', 'Away')
columns_to_drop = ['isHome', 'player.fieldTranslations.nameTranslation.ar',
                   'player.fieldTranslations.shortNameTranslation.ar','player.firstName',
                   'player.lastName','player.slug','player.shortName']
df_players.drop(columns=columns_to_drop, inplace=True)

# Ensure 'Team' is included only once in the list of columns
unique_columns = ['player.name', 'Team'] + [col for col in df_players.columns if col not in ['player.name', 'Team']]
df_players = df_players.reindex(columns=unique_columns)

# Sort the DataFrame by 'Team' to have 'Home' teams listed first
df_players_sorted = df_players.sort_values(by='Team', ascending=False)

# Fill NaN values with 0
df_players_sorted.fillna(0, inplace=True)
df_players_sorted.reset_index(drop=True, inplace=True)


df_players_sorted

,player.name,Team,shotType,situation,bodyPart,goalMouthLocation,xg,id,time,addedTime,...,draw.start.x,draw.start.y,draw.block.x,draw.block.y,draw.end.x,draw.end.y,draw.goal.x,draw.goal.y,xgot,goalType
0,Lamine Yamal,Home,miss,assisted,left-foot,close-high,0.016396,3433358,81,0.0,...,68.0,20.1,0.0,0.0,44.9,0,44.9,41.7,0.0000,0
1,Lamine Yamal,Home,block,fast-break,left-foot,low-left,0.036124,3433312,40,0.0,...,63.6,20.6,62.3,18.9,47.6,0,47.6,81.0,0.0000,0
2,Fabián Ruiz,Home,block,assisted,left-foot,low-centre,0.075778,3433292,36,0.0,...,48.9,18.6,48.9,12.4,49.0,0,49.0,81.0,0.0000,0
3,Dani Olmo,Home,goal,regular,right-foot,low-centre,0.122144,3433265,25,0.0,...,65.4,8.0,0.0,0.0,50.3,0,50.3,84.2,0.0282,regular
4,Lamine Yamal,Home,goal,assisted,left-foot,low-left,0.023169,3433264,21,0.0,...,62.7,24.6,0.0,0.0,46.1,0,46.1,81.0,0.3761,regular
5,Fabián Ruiz,Home,miss,assisted,head,close-high,0.539665,3433260,5,0.0,...,44.7,2.4,0.0,0.0,52.1,0,52.1,41.7,0.0000,0
6,Antoine Griezmann,Away,block,assisted,head,low-centre,0.028130,3433360,90,5.0,...,58.3,10.5,57.2,9.0,50.6,0,50.6,81.0,0.0000,0
7,Kylian Mbappé,Away,miss,assisted,right-foot,high,0.079102,3433359,86,0.0,...,39.0,14.1,0.0,0.0,45.4,0,45.4,33.3,0.0000,0
8,Theo Hernández,Away,miss,regular,right-foot,high,0.135995,3433356,76,0.0,...,44.8,15.1,0.0,0.0,50.6,0,50.6,27.8,0.0000,0
9,Dayot Upamecano,Away,miss,corner,head,right,0.126558,3433336,63,0.0,...,57.7,4.5,0.0,0.0,59.3,0,59.3,72.2,0.0000,0


In [14]:
df_players_sorted.to_csv('spain-france_shotmap.csv', index=False)